In [74]:
import pandas as pd

In [147]:
df = pd.read_csv('/volume_dir/boatrace/df_20160401-20230123.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [85]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 640

In [151]:
def preprocesser(df):
    # race_timeのないものを削除
    df = df[~(df['race_time']=='.')]
    df = df[~(df['race_time']=='..')]
    # float型に変更
    df["race_time"] = df["race_time"].map(lambda x: float(x[0])*60 + float(x[2:6]))
    # st_timeのないものを削除
    df = df[~(df['st_time']=='.')]
    df = df[~(df['st_time']=='..')]
    # float型に変更
    df['st_time']=df['st_time'].map(lambda x: float(x))
    # list内の列を偏差値に変更
    list_std = ['win_p','win_p_2','a_win_p','a_win_p_2','motor_p_2','boat_p_2']
    for list in list_std:    
        df_std = df[list].astype(float)
        win_mean=df_std.mean()
        win_std=df_std.std()
        if win_std==0: #もしdf_stdが同じ値だとwin_stdが0になってしまう
            df[list]=50.0
        else:
            df[list] = df_std.apply(lambda x : ((x - win_mean)*10/win_std+50))
        # ラベルエンコーディング　階級
    class_mapping = {'B2':1,'B1':2,'A2':3,'A1':4}
    df['class'] = df['class'].map(class_mapping)
    import sklearn.preprocessing
    #ラベルエンコード
    list_LabelEncode = ['place','type', 'weather', 'wind','name','area']
    for label in list_LabelEncode:
        le = sklearn.preprocessing.LabelEncoder()
        le.fit(df[label])
        df[label] = le.transform(df[label])
    df['race_date'] = pd.to_datetime(df['race_date'])
    
    return df
    
    

In [152]:
preprocesser(df)

,race_code,date,race_date,place,round,order,number,display,in,st_time,race_time,type,weather,wind,wind_strong,wave,title,race_name,position,name,age,area,weight,class,win_p,win_p_2,a_win_p,a_win_p_2,motor,motor_p_2,boat,boat_p_2,day,day_of_year,day_of_year_cos,day_of_year_sin
0,20191126KRT01R,5,2019-11-26,6,1,1,4189,6.71,1,0.11,109.4,5,0,1,3,3,スポーツニッポン杯,一般,1,797,38,13,56,4,56.353772,58.849463,58.714700,59.259681,36,42.479058,83,43.762367,26,330,0.815028,-0.579421
1,20191126KRT01R,5,2019-11-26,6,1,2,4485,6.71,3,0.15,110.4,5,0,1,3,3,スポーツニッポン杯,一般,3,1114,33,13,56,2,42.561856,40.851792,47.922174,42.815811,65,40.940378,72,49.739683,26,330,0.815028,-0.579421
2,20191126KRT01R,5,2019-11-26,6,1,3,3809,6.70,2,0.11,112.7,5,0,1,3,3,スポーツニッポン杯,一般,2,377,44,1,49,2,46.861688,46.126472,50.069940,50.659537,15,44.251133,81,54.688643,26,330,0.815028,-0.579421
3,20191126KRT01R,5,2019-11-26,6,1,4,4899,6.72,5,0.06,113.7,5,0,1,3,3,スポーツニッポン杯,一般,6,323,27,13,51,2,38.992183,41.438672,39.384803,39.411930,61,49.627869,87,52.861476,26,330,0.815028,-0.579421
4,20191126KRT01R,5,2019-11-26,6,1,5,4860,6.73,4,0.11,115.2,5,0,1,3,3,スポーツニッポン杯,一般,4,204,31,9,54,2,31.933967,31.715293,40.512380,41.527708,16,49.783466,90,52.374843,26,330,0.815028,-0.579421
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1706990,20170624EDG11R,2,2017-06-24,16,11,4,3709,7.05,5,0.19,118.0,2,1,5,5,5,男女Ｗ優勝戦 第４１回サンケイスポーツ杯,予選特選,5,1353,42,5,51,2,41.101535,44.061521,49.640387,46.279986,47,55.298510,65,61.905493,24,175,-0.990584,0.136906
1706993,20170624EDG12R,2,2017-06-24,16,12,1,3302,6.80,1,0.08,113.1,5,1,5,5,5,男女Ｗ優勝戦 第４１回サンケイスポーツ杯,記者選抜,1,1620,49,9,46,3,53.595389,54.734053,62.795456,59.259681,26,64.115665,46,44.450998,24,175,-0.990584,0.136906
1706994,20170624EDG12R,2,2017-06-24,16,12,2,4014,6.87,2,0.08,116.5,5,1,5,5,5,男女Ｗ優勝戦 第４１回サンケイスポーツ杯,記者選抜,2,1324,37,5,48,3,48.646525,46.488744,48.781280,52.046303,49,40.871224,27,47.793153,24,175,-0.990584,0.136906
1706995,20170624EDG12R,2,2017-06-24,16,12,3,3900,6.89,6,0.25,117.1,5,1,5,5,5,男女Ｗ優勝戦 第４１回サンケイスポーツ杯,記者選抜,6,1784,40,11,48,4,56.678288,55.357161,58.124064,59.259681,20,41.848026,38,60.665958,24,175,-0.990584,0.136906


## 決定木系のmodelを使用するため、label encodingする

In [142]:
# カテゴリ　'area' 18こ　'wind' 9こ　'place' 25こ 'type' 6こ　'weather' 5こ　
df['weather'].value_counts()

晴     773503
曇り    390153
雨     129515
雪       4843
霧         12
Name: weather, dtype: int64

In [146]:
df.to_csv('df_label_encode.csv', index=False,encoding='utf-8')